<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 6: Использование Advantage Actor Critic (A2C) для моделирования робототехники с помощью PyBullet и Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail" width="50%"/>

В этом блокноте вы научитесь использовать A2C с двумя наборами робототехнических сред PyBullet и Panda-Gym.

С [PyBullet](https://github.com/bulletphysics/bullet3), мы собираемся **обучить робота двигаться**:
- `AntBulletEnv-v0` 🕸️ Точнее, паук (они говорят Муравей, но да ладно... это паук 😆 ) 🕸️

Затем, с [Panda-Gym](https://github.com/qgallouedec/panda-gym), мы собираемся **обучить роботизированную руку** (Франка Эмика Панда робот) выполнять задание:
- `Reach`: робот должен поместить свой конечный манипулятор в целевое положение.

После этого вы сможете **тренироваться в других средах робототехники**.